In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from src import tensorfi2 as tfi

# 1. 下载 Tatoeba 英法并只取前 1% 样本以加速实验
raw = load_dataset("tatoeba", lang1="en", lang2="fr", split="train[:1%]")  # Tatoeba 英法平行语料 :contentReference[oaicite:4]{index=4}

# 2. 初始化分词器
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# 3. 预处理：添加翻译前缀，tokenize，并将 padding token 转为 -100 以忽略
def preprocess(examples):
    inputs  = ["translate English to French: " + ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    mi = tokenizer(inputs,  max_length=64, truncation=True, padding="max_length")
    lbl = tokenizer(targets, max_length=64, truncation=True, padding="max_length").input_ids
    lbl = [[(t if t != tokenizer.pad_token_id else -100) for t in seq] for seq in lbl]
    mi["labels"] = lbl
    return mi

tokenized = raw.map(preprocess, batched=True, remove_columns=["translation"])  # 动态预处理 :contentReference[oaicite:5]{index=5}


In [2]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM

# 可定制的超参数
MODEL_CHECKPOINT = "t5-small"   # 可换成 t5-base、t5-large 等 :contentReference[oaicite:6]{index=6}
LEARNING_RATE     = 5e-5
NUM_BEAMS         = 4

# 加载模型
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)  # 加载预训练 T5 编码器-解码器 :contentReference[oaicite:7]{index=7}

# 若需调整层数 / 头数，可在此处重新定义 config
# e.g., model.config.num_layers = 4; model.config.num_heads = 8


2025-05-02 02:43:59.893317: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-05-02 02:43:59.893339: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-05-02 02:43:59.893346: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1746125039.893355  523621 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1746125039.893371  523621 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can 

In [3]:
tf_train = tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    batch_size=16,
    shuffle=True
)
tf_test = tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    batch_size=16,
    shuffle=False
)

model.compile(optimizer="adam")  # 不需要显式传入 loss
# 假设 model 是您搭建好的 Transformer（tf.keras.Model）
model.summary()
model.fit(tf_train, epochs=1, validation_data=tf_test)

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2025-05-02 02:44:05.791238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


166/166 [==============================] - 95s 523ms/step - loss: 0.9550 - val_loss: 0.4866


In [4]:
import logging
logging.basicConfig(level=logging.DEBUG)

results = model.evaluate(tf_test)              # 评估
print("Test loss:", results)

tfi.inject(model=model,
           confFile="/Users/lordtarn1shed/TensorFI2/experiments/layer-states/confFiles/sample.yaml",
           log_level="DEBUG")

results = model.evaluate(tf_test)              # 评估故障
print("Test loss:", results)

166/166 [==============================] - 25s 150ms/step - loss: 0.4866


DEBUG:root:Logging level set to DEBUG
INFO:root:Starting fault injection in user-specified layer 1
INFO:root:Completed injections... exiting


Test loss: 0.4865816831588745
Total trainable layers (variables): 131
[Layer 0] Name: shared/shared/embeddings:0, Shape: (32128, 512), Elements: 16449536
[Layer 1] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/relative_attention_bias/embeddings:0, Shape: (32, 8), Elements: 256
[Layer 2] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/q/kernel:0, Shape: (512, 512), Elements: 262144
[Layer 3] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/k/kernel:0, Shape: (512, 512), Elements: 262144
[Layer 4] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/v/kernel:0, Shape: (512, 512), Elements: 262144
[Layer 5] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/SelfAttention/o/kernel:0, Shape: (512, 512), Elements: 262144
[Layer 6] Name: tft5_for_conditional_generation/encoder/block_._0/layer_._0/layer_norm/weight:0, Shape: (512,), Elements: 512
[Layer 7] 

In [5]:
# 推理示例
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

sample = "translate English to French: This is a test."
inp = tokenizer(sample, return_tensors="tf", padding="max_length", max_length=64)
with tf.device('/CPU:0'):
    out = model.generate(inp["input_ids"], attention_mask=inp["attention_mask"], num_beams=NUM_BEAMS)
print("Translation:", tokenizer.decode(out[0], skip_special_tokens=True))

/Users/lordtarn1shed/anaconda3/envs/transformer/lib/python3.10/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
I0000 00:00:1746125185.463789  523621 service.cc:152] XLA service 0x6000000f4b00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746125185.463802  523621 service.cc:160]   StreamExecutor device (0): Host, Default Version
2025-05-02 02:46:25.465670: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746125185.495983  523621 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Translation: C'est un essai.
